In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from src.network import Network_custom
from src.printing import *
# G-code generation codes are cloned from https://github.com/tibor-barsi/GcodeGenerator. Tibor Barsi is the author of the code.
from src.g_code_generation_copy.gcode_generator import G_code_generator
from src.g_code_generation_copy.tool_changer_functions import load_params
from src.network import Network_custom
import os
import plotly.io as pio
pio.renderers.default = 'browser'

Define the unit cell network

In [ ]:
def Rot(theta):
    """
    Rotation matrix around the z-axis
    :param theta: angle in radians
    :return: rotation matrix
    """
    return np.array([[np.cos(theta), -np.sin(theta), 0],
                     [np.sin(theta), np.cos(theta), 0],
                     [0, 0, 1]])

# Note that the coordinates of the vertices will change after equilibrium is reached.
model_name = "validation_unit"
L_scale = 10.5
vertices    = np.array([[-10, 0, 0], [-5, 0,0], [0, 0, 0], [5, 0, 0], [10, 0, 0], [0,-10,0], [0,-5,0], [0,5,0], [0,10,0]])*L_scale  # The coordinates of the vertices in mm
vertices    = vertices@Rot(np.pi/4)
edges       = [[0, 1], [1, 2], [2, 3], [3, 4], [5, 6], [6, 2], [2, 7], [7, 8], [1, 6], [6, 3], [3, 7], [7,1]]                  # The edges of the network.
paths       = [[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11]]   # 3D printable paths. Ensure an edge is not flipped.
directions  = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]          # Flip the curvature of an edge by setting the direction to -1.
center_cross_path = [1, 2, 5, 6]
connector_path = [0, 3, 4, 7]
q_scalar = 1.4
sum_q0 = 0.02*q_scalar
sum_q1 = 0.02*q_scalar
sum_q2 = 0.01*q_scalar
offset = 1/32
ratio_0 = np.array([3/8-offset, 1/8, 1/8, 3/8+offset])
ratio_1 = np.array([3/8, 1/8-offset, 1/8+offset, 3/8])
ratio_2 = np.array([2/8-offset, 2/8+offset, 2/8, 2/8])

q = np.zeros(len(edges))
q[paths[0]] = sum_q0*ratio_0
q[paths[1]] = sum_q1*ratio_1
q[paths[2]] = sum_q2*ratio_2

fixed       = [0, 4, 5, 8]
# Create the network
net = Network_custom.from_fd(vertices, edges, q, fixed, paths = paths, dir = directions)
net.net_plot(color=True, elables = False, vlabels = True)
np.max(net.f)/0.078294515

In [ ]:
printing_params = load_params(r'DATA/NT_Eel_0.2mm.json')
alpha_loop = np.deg2rad(30) # The angle of the loop
L_loop  = 9                # The length of the loop
n_points = 30               # The number of points in the loop
net.n_split = n_points

start_gcode     = open(r'DATA/start_gcode_XL.gcode', 'r').read()
end_gcode       = open(r'DATA/end_gcode_Prusa.gcode', 'r').read()

temperature_settings = {'first_layer_bed_temperature': 60, 'first_layer_temperature':220, 'K-factor': 0.20, 'idle_temperature': printing_params["T_nozzle_standby"]}
comment = ''

bed_width = 230
bed_height = 210

start_loop_bools = [True, True, False]
end_loop_bools = [True, True, False]

running_start_bools = [True, True, True]
running_end_bools = [True, True, False]
L_running = 6

file_path = os.path.join('DATA', 'Avg_Stress_Strain_Overture_TPU.csv')
stress_data, strain_data = net.load_stress_strain_curve(file_path, A_scale = 1.0)

TPU_nl = {'stress':strain_data, 'strain': stress_data, 'v':0.3897, 'p':1.18e-9, 'A': 0.078294515, 'name': 'TPU Overture'} # TPU Overture non-conductive

A = [TPU_nl['A']]*len(net.edges)

# Purge line coors
gen = G_code_generator(printing_params)
point0, point1 = [20,15,printing_params['layer_height']-printing_params['first_layer_offset']], [150,15,printing_params['layer_height']-printing_params['first_layer_offset']]
point2, point3 = [150,10,printing_params['layer_height']-printing_params['first_layer_offset']], [20,10,printing_params['layer_height']-printing_params['first_layer_offset']]

Generate a Specimen

In [ ]:
reference_point = [0,0,0] 
l0, l_scalar = net.materialize_nonlinear(A, stress_data, strain_data, interpolation_kind = 'cubic')
net.initialize_shape_optimizer(function_type = 'standard',  method = 'Gauss-Seidel',options ={"maxiter": 1000, "damping": .01, "correction_scalar": 1., "tol": 1e-6})
net.optimize_vertices()
net.scale_vertices(reference_point, net.l_scalar, account_for_leafs = True) # If you don't provide a scalar, it will use network.l_scalar automatically
R, th = net.arc_param()
xyz = net.arc_points(n = n_points)
net.auto_flip_curves(n = n_points)        # Automatically flip the curvature of the edges. Directions will become 1, -1, 1, -1, ...
net.flip_curves(n = n_points)                   # Flip all the edges.
net.flip_curves([2, 5, 8], n = n_points)
net.jump_at_intersection(intersection_width = printing_params['d_nozzle']*1.5, intersection_height = printing_params['layer_height'], interpolation_function=None) 
net.net_plot(plot_type = 'arcs')
net.all_loop_to_path(start_loop_bools, end_loop_bools, L_loop, alpha_loop, n_points)
net.add_running_start(running_start_bools, running_end_bools, L_running, n_points)
file_name = model_name + f'_0'

g_code = replace_brackets(start_gcode, temperature_settings)
g_code += make_purge_line(printing_params, gen, point0, point1, comment='Purge line', ef_extra=1.5)
g_code += make_purge_line(printing_params, gen, point2, point3, comment='Purge line', ef_extra=1.05)

for path_i, cor_list in enumerate(net.paths_xyz):
    cor_list[:,0] += bed_width/2 
    cor_list[:,1] += bed_height/2 
    g_code += add_path_to_gcode(cor_list, gen, printing_params, comment = 'Path (' + str(path_i) + ')')
# g_code += add_net_to_gcode(printing_params, gen, net)

g_code += end_gcode
save_path = os.path.join('DATA', 'generated_gcodes', model_name + '.gcode')
os.makedirs(os.path.dirname(save_path), exist_ok=True)
save_gcode(save_path, g_code)
print('G-code generated')